In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
from pytz import timezone
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from multiprocessing import Process, Queue
from functools import partial
import multiprocessing
from multiprocessing import Pool

In [2]:
from functools import partial

In [3]:
pd.set_option('display.max_columns', None)

In [88]:
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [89]:
onchainDF_tickers = pd.read_csv("../data/train/tickers_train.csv")

In [90]:
onchainDF_tickers.head()

,Unnamed: 0,datetime,name,priceBtc,priceUsd,ticker,volume
0,0,2018-03-01T00:00:00Z,0x,0.000087,0.902278,ZRX,8127100.0
1,1,2018-03-01T02:00:00Z,0x,0.000086,0.895107,ZRX,7962720.0
2,2,2018-03-01T04:00:00Z,0x,0.000088,0.917095,ZRX,7980440.0
3,3,2018-03-01T06:00:00Z,0x,0.000087,0.910295,ZRX,7554380.0
4,4,2018-03-01T08:00:00Z,0x,0.000087,0.927460,ZRX,7468790.0


In [91]:
# check the ost active tickers in ochain
onchainDF_tickers.groupby("ticker")["name"].count().nlargest()

ticker
EDR      1464
HOT      1464
SMART    1464
LNC       824
1ST       732
Name: name, dtype: int64

In [92]:
onchainDF_tickers.shape

(322421, 7)

In [93]:
onchainDF_tickers.isnull().sum()

Unnamed: 0    0
datetime      0
name          0
priceBtc      0
priceUsd      0
ticker        0
volume        1
dtype: int64

In [94]:
onchainDF_tickers[onchainDF_tickers["ticker"] == "DRGN"].shape

(732, 7)

In [95]:
#onchainDF_DRGN = pd.read_pickle("../data/train/onchain/DRGN_train_withDates.pickle")
onchainDF_all = pd.read_pickle("../data/train/onchain/onchain_train_withDates.pickle")

In [110]:
onchainDF_all.head()

,Unnamed: 0,datetime,name,priceBtc,priceUsd,ticker,volume,date_date,date_datetime,date_string
0,0,2018-03-01T00:00:00Z,0x,0.000087,0.902278,ZRX,8127100.0,2018-03-01,2018-03-01,2018-03-01 00:00:00
200879,0,2018-03-01T00:00:00Z,NAGA,0.000091,0.938037,NGC,295938.0,2018-03-01,2018-03-01,2018-03-01 00:00:00
200147,0,2018-03-01T00:00:00Z,Mysterium,0.000063,0.652940,MYST,163384.0,2018-03-01,2018-03-01,2018-03-01 00:00:00
199415,0,2018-03-01T00:00:00Z,MyBit Token,0.000481,4.971757,MYB,17612.0,2018-03-01,2018-03-01,2018-03-01 00:00:00
198791,0,2018-03-01T00:00:00Z,Musiconomi,0.000009,0.090728,MCI,386.0,2018-03-01,2018-03-01,2018-03-01 00:00:00


In [97]:
onchainDF_all.shape

(322421, 10)

In [98]:
onchainDF_all.isnull().sum()

Unnamed: 0       0
datetime         0
name             0
priceBtc         0
priceUsd         0
ticker           0
volume           1
date_date        0
date_datetime    0
date_string      0
dtype: int64

In [99]:
onchainDF_all = onchainDF_all.sort_values('date_datetime',ascending=True)

In [100]:
min_date = datetime.strptime("2018-03-01 00:00:00","%Y-%m-%d %H:%M:%S")

In [101]:
min_date, max_date

(datetime.datetime(2018, 3, 1, 0, 0), Timestamp('2018-05-01 02:58:10'))

In [200]:
def calc_stats_ticker_market(df):
    #min_date = df.iloc[0]["date_datetime"]
    #min_date = datetime.strptime("%Y-%m-%dT%H:%M:%SZ", "2018-03-01 00:00:00")
    ticker_list = onchainDF_all['ticker'].unique()
    min_date = datetime.strptime("2018-03-01 00:00:00","%Y-%m-%d %H:%M:%S")
    max_date = df.iloc[-1]["date_datetime"]
    delta = timedelta(hours=2)
    cur_date = min_date
    stats_list = []
    i = 1
    init_dict = {el+"_usd":0 for el in ticker_list}
    while cur_date <= max_date:
        next_date = cur_date + delta
        tmpGroup = df[(df['date_datetime'] >= cur_date) & (df['date_datetime']<next_date)].copy()
        print cur_date, next_date, tmpGroup.shape
        slotStatDict = init_dict.copy()
        slotStatDict['start_date'] = cur_date
        slotStatDict['end_date'] = next_date
        slotStatDict['sum_all_usd'] = (tmpGroup['volume']*tmpGroup['priceUsd']).sum()
        slotStatDict['sum_all_btc'] = (tmpGroup['volume']*tmpGroup['priceBtc']).sum()
        tmpGroup_byticker = tmpGroup.groupby('ticker')
        for name, group in tmpGroup_byticker:
            key_name = name + '_usd'
            slotStatDict[key_name] = (group['volume']*group['priceUsd']).sum()
        cur_date = cur_date + delta
        stats_list.append(slotStatDict)
        i = i +1
        #if i>3:
            #break
    return stats_list

In [201]:
%%time
ll = calc_stats_ticker_market(onchainDF_all)

2018-03-01 00:00:00 2018-03-01 02:00:00 (378, 10)
2018-03-01 02:00:00 2018-03-01 04:00:00 (378, 10)
2018-03-01 04:00:00 2018-03-01 06:00:00 (378, 10)
2018-03-01 06:00:00 2018-03-01 08:00:00 (378, 10)
2018-03-01 08:00:00 2018-03-01 10:00:00 (378, 10)
2018-03-01 10:00:00 2018-03-01 12:00:00 (378, 10)
2018-03-01 12:00:00 2018-03-01 14:00:00 (378, 10)
2018-03-01 14:00:00 2018-03-01 16:00:00 (378, 10)
2018-03-01 16:00:00 2018-03-01 18:00:00 (378, 10)
2018-03-01 18:00:00 2018-03-01 20:00:00 (378, 10)
2018-03-01 20:00:00 2018-03-01 22:00:00 (382, 10)
2018-03-01 22:00:00 2018-03-02 00:00:00 (382, 10)
2018-03-02 00:00:00 2018-03-02 02:00:00 (382, 10)
2018-03-02 02:00:00 2018-03-02 04:00:00 (382, 10)
2018-03-02 04:00:00 2018-03-02 06:00:00 (382, 10)
2018-03-02 06:00:00 2018-03-02 08:00:00 (382, 10)
2018-03-02 08:00:00 2018-03-02 10:00:00 (382, 10)
2018-03-02 10:00:00 2018-03-02 12:00:00 (382, 10)
2018-03-02 12:00:00 2018-03-02 14:00:00 (382, 10)
2018-03-02 14:00:00 2018-03-02 16:00:00 (381, 10)


2018-03-14 16:00:00 2018-03-14 18:00:00 (410, 10)
2018-03-14 18:00:00 2018-03-14 20:00:00 (410, 10)
2018-03-14 20:00:00 2018-03-14 22:00:00 (411, 10)
2018-03-14 22:00:00 2018-03-15 00:00:00 (412, 10)
2018-03-15 00:00:00 2018-03-15 02:00:00 (411, 10)
2018-03-15 02:00:00 2018-03-15 04:00:00 (411, 10)
2018-03-15 04:00:00 2018-03-15 06:00:00 (411, 10)
2018-03-15 06:00:00 2018-03-15 08:00:00 (411, 10)
2018-03-15 08:00:00 2018-03-15 10:00:00 (411, 10)
2018-03-15 10:00:00 2018-03-15 12:00:00 (411, 10)
2018-03-15 12:00:00 2018-03-15 14:00:00 (412, 10)
2018-03-15 14:00:00 2018-03-15 16:00:00 (411, 10)
2018-03-15 16:00:00 2018-03-15 18:00:00 (411, 10)
2018-03-15 18:00:00 2018-03-15 20:00:00 (411, 10)
2018-03-15 20:00:00 2018-03-15 22:00:00 (411, 10)
2018-03-15 22:00:00 2018-03-16 00:00:00 (410, 10)
2018-03-16 00:00:00 2018-03-16 02:00:00 (410, 10)
2018-03-16 02:00:00 2018-03-16 04:00:00 (410, 10)
2018-03-16 04:00:00 2018-03-16 06:00:00 (410, 10)
2018-03-16 06:00:00 2018-03-16 08:00:00 (410, 10)


2018-03-28 08:00:00 2018-03-28 10:00:00 (437, 10)
2018-03-28 10:00:00 2018-03-28 12:00:00 (436, 10)
2018-03-28 12:00:00 2018-03-28 14:00:00 (436, 10)
2018-03-28 14:00:00 2018-03-28 16:00:00 (436, 10)
2018-03-28 16:00:00 2018-03-28 18:00:00 (437, 10)
2018-03-28 18:00:00 2018-03-28 20:00:00 (438, 10)
2018-03-28 20:00:00 2018-03-28 22:00:00 (438, 10)
2018-03-28 22:00:00 2018-03-29 00:00:00 (438, 10)
2018-03-29 00:00:00 2018-03-29 02:00:00 (438, 10)
2018-03-29 02:00:00 2018-03-29 04:00:00 (438, 10)
2018-03-29 04:00:00 2018-03-29 06:00:00 (437, 10)
2018-03-29 06:00:00 2018-03-29 08:00:00 (438, 10)
2018-03-29 08:00:00 2018-03-29 10:00:00 (437, 10)
2018-03-29 10:00:00 2018-03-29 12:00:00 (438, 10)
2018-03-29 12:00:00 2018-03-29 14:00:00 (438, 10)
2018-03-29 14:00:00 2018-03-29 16:00:00 (438, 10)
2018-03-29 16:00:00 2018-03-29 18:00:00 (438, 10)
2018-03-29 18:00:00 2018-03-29 20:00:00 (438, 10)
2018-03-29 20:00:00 2018-03-29 22:00:00 (438, 10)
2018-03-29 22:00:00 2018-03-30 00:00:00 (438, 10)


2018-04-11 00:00:00 2018-04-11 02:00:00 (459, 10)
2018-04-11 02:00:00 2018-04-11 04:00:00 (459, 10)
2018-04-11 04:00:00 2018-04-11 06:00:00 (459, 10)
2018-04-11 06:00:00 2018-04-11 08:00:00 (459, 10)
2018-04-11 08:00:00 2018-04-11 10:00:00 (459, 10)
2018-04-11 10:00:00 2018-04-11 12:00:00 (459, 10)
2018-04-11 12:00:00 2018-04-11 14:00:00 (459, 10)
2018-04-11 14:00:00 2018-04-11 16:00:00 (459, 10)
2018-04-11 16:00:00 2018-04-11 18:00:00 (460, 10)
2018-04-11 18:00:00 2018-04-11 20:00:00 (460, 10)
2018-04-11 20:00:00 2018-04-11 22:00:00 (459, 10)
2018-04-11 22:00:00 2018-04-12 00:00:00 (459, 10)
2018-04-12 00:00:00 2018-04-12 02:00:00 (460, 10)
2018-04-12 02:00:00 2018-04-12 04:00:00 (460, 10)
2018-04-12 04:00:00 2018-04-12 06:00:00 (460, 10)
2018-04-12 06:00:00 2018-04-12 08:00:00 (460, 10)
2018-04-12 08:00:00 2018-04-12 10:00:00 (459, 10)
2018-04-12 10:00:00 2018-04-12 12:00:00 (459, 10)
2018-04-12 12:00:00 2018-04-12 14:00:00 (460, 10)
2018-04-12 14:00:00 2018-04-12 16:00:00 (459, 10)


2018-04-24 16:00:00 2018-04-24 18:00:00 (498, 10)
2018-04-24 18:00:00 2018-04-24 20:00:00 (498, 10)
2018-04-24 20:00:00 2018-04-24 22:00:00 (498, 10)
2018-04-24 22:00:00 2018-04-25 00:00:00 (498, 10)
2018-04-25 00:00:00 2018-04-25 02:00:00 (498, 10)
2018-04-25 02:00:00 2018-04-25 04:00:00 (498, 10)
2018-04-25 04:00:00 2018-04-25 06:00:00 (498, 10)
2018-04-25 06:00:00 2018-04-25 08:00:00 (498, 10)
2018-04-25 08:00:00 2018-04-25 10:00:00 (498, 10)
2018-04-25 10:00:00 2018-04-25 12:00:00 (498, 10)
2018-04-25 12:00:00 2018-04-25 14:00:00 (498, 10)
2018-04-25 14:00:00 2018-04-25 16:00:00 (497, 10)
2018-04-25 16:00:00 2018-04-25 18:00:00 (498, 10)
2018-04-25 18:00:00 2018-04-25 20:00:00 (498, 10)
2018-04-25 20:00:00 2018-04-25 22:00:00 (498, 10)
2018-04-25 22:00:00 2018-04-26 00:00:00 (503, 10)
2018-04-26 00:00:00 2018-04-26 02:00:00 (503, 10)
2018-04-26 02:00:00 2018-04-26 04:00:00 (503, 10)
2018-04-26 04:00:00 2018-04-26 06:00:00 (503, 10)
2018-04-26 06:00:00 2018-04-26 08:00:00 (502, 10)


In [194]:
len(ll)

3

In [195]:
#ll[2]

In [202]:
all_ticks_df = pd.DataFrame(ll)

In [206]:
all_ticks_df.head()

,$PAC_usd,1ST_usd,1WO_usd,ABT_usd,ACAT_usd,ACC_usd,ACT_usd,ADA_usd,ADB_usd,ADH_usd,ADI_usd,ADST_usd,ADX_usd,AEON_usd,AE_usd,AGI_usd,AID_usd,AION_usd,AIR_usd,AIX_usd,AMB_usd,AMLT_usd,ANT_usd,APPC_usd,ARDR_usd,ARK_usd,ARN_usd,ART_usd,ARY_usd,AST_usd,ATC_usd,ATL_usd,ATN_usd,ATX_usd,AUC_usd,AUTO_usd,AVH_usd,AVT_usd,AXP_usd,BANCA_usd,BAT_usd,BAX_usd,BAY_usd,BBC_usd,BBI_usd,BCA_usd,BCC_usd,BCD_usd,BCH_usd,BCN_usd,BCO_usd,BEE_usd,BERRY_usd,BET_usd,BFT_usd,BIO_usd,BIS_usd,BIX_usd,BKX_usd,BLOCK_usd,BLT_usd,BLZ_usd,BNB_usd,BNT_usd,BOS_usd,BOT_usd,BPT_usd,BRD_usd,BRM_usd,BSM_usd,BTCD_usd,BTCP_usd,BTC_usd,BTG_usd,BTM_usd,BTO_usd,BTS_usd,BTX_usd,BUBO_usd,BURST_usd,C20_usd,CAS_usd,CBT_usd,CENNZ_usd,CFI_usd,CHP_usd,CHSB_usd,CHX_usd,CLOAK_usd,CLR_usd,CMT_usd,CND_usd,CNX_usd,COB_usd,COFI_usd,COSS_usd,COV_usd,CPC_usd,CPX_usd,CPY_usd,CRC_usd,CROP_usd,CRPT_usd,CSC_usd,CS_usd,CTR_usd,CTXC_usd,CVCOIN_usd,CVC_usd,CVH_usd,CVT_usd,CV_usd,CXO_usd,DADI_usd,DAI_usd,DAN_usd,DASH_usd,DATA_usd,DATX_usd,DAY_usd,DBC_usd,DBET_usd,DCN_usd,DCR_usd,DDD_usd,DEB_usd,DENT_usd,DERO_usd,DEV_usd,DEW_usd,DGB_usd,DGD_usd,DGPT_usd,DICE_usd,DIG_usd,DLT_usd,DML_usd,DMT_usd,DNA_usd,DNT_usd,DOCK_usd,DOGE_usd,DPY_usd,DRGN_usd,DRG_usd,DROP_usd,DRPU_usd,DTA_usd,DTH_usd,DTR_usd,DXT_usd,EARTH_usd,ECA_usd,ECC_usd,ECH_usd,ECN_usd,EDG_usd,EDO_usd,EDR_usd,EDT_usd,EFX_usd,EGC_usd,EKT_usd,ELA_usd,ELEC_usd,ELF_usd,EMC2_usd,EMC_usd,ENG_usd,ENJ_usd,EOSDAC_usd,EOS_usd,ERA_usd,ESZ_usd,ETC_usd,ETHOS_usd,ETH_usd,ETN_usd,ETP_usd,EVE_usd,EVN_usd,EXP_usd,EXY_usd,EZT_usd,FCT_usd,FDZ_usd,FID_usd,FLO_usd,FLUZ_usd,FOTA_usd,FSN_usd,FTC_usd,FTX_usd,FUN_usd,GAME_usd,GAS_usd,GAT_usd,GBYTE_usd,GEM_usd,GJC_usd,GLA_usd,GNO_usd,GNT_usd,GNX_usd,GRE_usd,GRFT_usd,GRMD_usd,GRS_usd,GRX_usd,GTC_usd,GTO_usd,GUESS_usd,GUP_usd,GVT_usd,GXS_usd,HAV_usd,HBZ_usd,HDG_usd,HERO_usd,HGT_usd,HIRE_usd,HMQ_usd,HOT_usd,HPB_usd,HQX_usd,HSR_usd,HTML_usd,HT_usd,ICN_usd,ICX_usd,IDH_usd,IFC_usd,IFT_usd,IGNIS_usd,IHT_usd,INDI_usd,IND_usd,ING_usd,INK_usd,INSTAR_usd,INS_usd,INT_usd,ION_usd,IOST_usd,IPL_usd,IPSX_usd,ITC_usd,J8T_usd,JC_usd,JEW_usd,JNT_usd,KCS_usd,KEY_usd,KIN_usd,KMD_usd,KNC_usd,KST_usd,LALA_usd,LBC_usd,LCC_usd,LDC_usd,LEDU_usd,LEND_usd,LGO_usd,LINK_usd,LKK_usd,LNC_usd,LOCI_usd,LOC_usd,LOOM_usd,LRC_usd,LSK_usd,LST_usd,LTC_usd,LUN_usd,LYM_usd,MAGE_usd,MAID_usd,MANA_usd,MAN_usd,MCI_usd,MCO_usd,MDS_usd,MED_usd,MFG_usd,MGO_usd,MIOTA_usd,MITH_usd,MKR_usd,MLM_usd,MLN_usd,MNTP_usd,MNX_usd,MOAC_usd,MOD_usd,MONA_usd,MRK_usd,MSR_usd,MTH_usd,MTL_usd,MTN_usd,MTX_usd,MWAT_usd,MYB_usd,MYST_usd,NANJ_usd,NANO_usd,NAS_usd,NAV_usd,NCASH_usd,NCT_usd,NEBL_usd,NEO_usd,NGC_usd,NLG_usd,NOAH_usd,NOTE_usd,NPXS_usd,NPX_usd,NTK_usd,NULS_usd,NXS_usd,NXT_usd,OAX_usd,OCN_usd,OC_usd,ODE_usd,OMG_usd,ONT_usd,ORI_usd,ORME_usd,OST_usd,PARETO_usd,PART_usd,PAYX_usd,PAY_usd,PCL_usd,PGL_usd,PHI_usd,PIVX_usd,PKT_usd,PLAY_usd,PLBT_usd,PLR_usd,POA_usd,POE_usd,POLY_usd,POWR_usd,PPC_usd,PPP_usd,PPT_usd,PRE_usd,PRIX_usd,PRL_usd,PRO_usd,PST_usd,PURA_usd,PXC_usd,QASH_usd,QRL_usd,QSP_usd,QTUM_usd,RCN_usd,RDD_usd,RDN_usd,RED_usd,REM_usd,REN_usd,REP_usd,REQ_usd,RFR_usd,RHOC_usd,RKT_usd,RLC_usd,RMT_usd,RNTB_usd,RPX_usd,RUFF_usd,RVN_usd,R_usd,SAFEX_usd,SALT_usd,SAN_usd,SBD_usd,SCC_usd,SC_usd,SENC_usd,SENT_usd,SEN_usd,SETH_usd,SHIP_usd,SHP_usd,SIG_usd,SKY_usd,SLR_usd,SLS_usd,SLT_usd,SMART_usd,SMS_usd,SMT_usd,SNGLS_usd,SNIP_usd,SNM_usd,SNT_usd,SOAR_usd,SOC_usd,SPANK_usd,SPD_usd,SPF_usd,SPHTX_usd,SRCOIN_usd,SRN_usd,STAR_usd,STEEM_usd,STK_usd,STORJ_usd,STORM_usd,STQ_usd,STRAT_usd,STU_usd,STX_usd,SUB_usd,SUMO_usd,SUP_usd,SWH_usd,SWM_usd,SXDT_usd,SYS_usd,TAAS_usd,TAU_usd,TBAR_usd,TBX_usd,TCC_usd,TDS_usd,TEL_usd,TEN_usd,TFD_usd,THETA_usd,TIO_usd,TKY_usd,TNB_usd,TNC_usd,TNT_usd,TOMO_usd,TPAY_usd,TRAC_usd,TRAK_usd,TRF_usd,TRST_usd,TRUE_usd,TRX_usd,TUSD_usd,UBQ_usd,UCASH_usd,UP_usd,USDT_usd,UTK_usd,UTNP_usd,UUU_usd,VEE_usd,VEN_usd,VERI_usd,VIA_usd,VIBE_usd,VIT_usd,VLC_usd,VSL_usd,VTC_usd,W3C_usd,WABI_usd,WAN_usd,WAVES_usd,WAX_usd,WICC_usd,WINGS_usd,WIN_usd,WPR_usd,WTC_usd,XAS_usd,XBP

In [205]:
all_ticks_df.to_pickle("../pickles/onchain_features.pickle")